### Import libraries

In [1]:
import numpy as np
import pandas as pd
from absl import flags
import tensorflow as tf
import tensorflow_privacy
from keras.models import Model
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
from keras.layers.convolutional import Conv2D
from tensorflow.keras.datasets import cifar10
from tensorflow_addons.layers import GroupNormalization
from keras.layers import Input, Dense, Flatten, Dropout, Add
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.analysis.rdp_accountant import compute_rdp
from tensorflow_privacy.privacy.analysis.rdp_accountant import get_privacy_spent
from keras.layers.pooling import MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
tf.compat.v1.disable_v2_behavior()
tf.get_logger().setLevel('ERROR')
%matplotlib inline

Instructions for updating:
non-resource variables are not supported in the long term


### Load Dataset

In [2]:
# Loading cifar10 image dataset
data_train, data_test = cifar10.load_data()
x_train, y_train = data_train
x_test, y_test = data_test

# Normalizing pixel values of images
x_train = x_train / 255.0
x_test = x_test / 255.0

# Flattening images in dataset
y_train = y_train.flatten().reshape(y_train.shape[0], 1)
y_test = y_test.flatten().reshape(y_test.shape[0], 1)

# One hot encoding of labels/target column
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

print("training:", x_train.shape, y_train.shape)
print("testing:", x_test.shape, y_test.shape)

training: (50000, 32, 32, 3) (50000, 10)
testing: (10000, 32, 32, 3) (10000, 10)


### Model5: Custom Simple CNN

In [12]:
# Hyper-parameters
epochs = 41
batch_size = 1500
l2_norm_clip = 1
noise_multiplier = 1.3
num_microbatches = 100
learning_rate = 0.15
delta = 1e-5

# epochs = 20
# batch_size = 500
# l2_norm_clip = 1.5
# noise_multiplier = 1.3
# num_microbatches = 250
# learning_rate = 0.25

if batch_size % num_microbatches != 0:
    raise ValueError('Batch size should be an integer multiple of the number of microbatches')
    
# Compute RDP
orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
rdp = compute_rdp(q = batch_size / 50000,
                  noise_multiplier = noise_multiplier,
                  steps = epochs * 50000 // batch_size,
                  orders = orders)
# Calculate epsilon
epsilon = get_privacy_spent(orders, rdp, target_delta = delta)[0]
epsilon

4.979053549560634

In [4]:
### Custom ResNet model
model_input = Input(shape = (32, 32, 3))
x1 = Conv2D(16, (8, 8), activation = 'tanh', strides = 2)(model_input)
x2 = MaxPooling2D(pool_size = (2,1))(x1)
x3 = Conv2D(32, (4, 4), activation = 'tanh', strides = 2)(x2)
x4 = MaxPooling2D(pool_size = (2,1))(x3)

# ## Residual Block 1
# x3 = Conv2D(64, (3, 3), activation = 'relu', strides = 1)(x2)
# x3 = GroupNormalization(groups = 64, axis=3)(x3) # normalization layer 1
# x4 = Conv2D(64, (3, 3), activation = 'relu', strides = 1)(x3)
# sk1 = Add()([x2, x4]) # skip connection 1
# ## Residual Block 2
# x5 = Conv2D(64, (3, 3), activation = 'relu', strides = 1)(sk1)
# x5 = GroupNormalization(groups = 64, axis=3)(x5) # normalization layer 2
# x6 = Conv2D(64, (3, 3), activation = 'relu', strides = 1)(x5)
# sk2 = Add()([sk1, x6]) # skip connection 2
# Global Average Pooling layer
# avg = GlobalAveragePooling2D()(sk2)

ftn = Flatten()(x4)
# Feed-forward network
d1 = Dense(64, activation = 'tanh')(ftn)
d2 = Dense(32, activation = 'tanh')(d1)
model_output = Dense(10, activation = 'softmax')(d2)
model = Model(inputs = model_input, outputs = model_output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 13, 13, 16)        3088      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 6, 13, 16)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 2, 5, 32)          8224      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 5, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 160)               0     

In [5]:
# define optimizer (dp-sgd) 
optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)

# # define optimizer (dp-adam)
# optimizer = tensorflow_privacy.DPKerasAdamOptimizer(
#     l2_norm_clip=l2_norm_clip,
#     noise_multiplier=noise_multiplier,
#     num_microbatches=num_microbatches,
#     learning_rate=learning_rate,
#     gradient_accumulation_steps=5)

# define loss function
loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

In [6]:
# Compile model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
# Train model
model.fit(x_train, y_train,
          epochs=epochs,
          validation_data=(x_test, y_test),
          batch_size=batch_size)

/Users/Mahos/miniforge3/envs/shm_tf/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


Train on 50000 samples, validate on 10000 samples
Metal device set to: Apple M1 Pro


2022-03-24 19:05:09.869095: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-24 19:05:09.869235: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-03-24 19:05:09.995101: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-24 19:05:09.997151: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1/130


2022-03-24 19:05:10.141793: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-24 19:05:10.279402: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-24 19:05:10.311692: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50000/50000 [==============================] - ETA: 0s - loss: 2.1793 - acc: 0.1970

/Users/Mahos/miniforge3/envs/shm_tf/lib/python3.9/site-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
2022-03-24 19:08:18.056319: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50000/50000 [==============================] - 188s 4ms/sample - loss: 2.1793 - acc: 0.1970 - val_loss: 2.0698 - val_acc: 0.2355
Epoch 2/130
50000/50000 [==============================] - 175s 4ms/sample - loss: 2.0320 - acc: 0.2459 - val_loss: 1.9646 - val_acc: 0.2756
Epoch 3/130
50000/50000 [==============================] - 172s 3ms/sample - loss: 1.9627 - acc: 0.2766 - val_loss: 1.9437 - val_acc: 0.2852
Epoch 4/130
50000/50000 [==============================] - 173s 3ms/sample - loss: 1.9127 - acc: 0.2981 - val_loss: 1.8665 - val_acc: 0.3164
Epoch 5/130
50000/50000 [==============================] - 175s 4ms/sample - loss: 1.8628 - acc: 0.3193 - val_loss: 1.8327 - val_acc: 0.3315
Epoch 6/130
50000/50000 [==============================] - 204s 4ms/sample - loss: 1.8287 - acc: 0.3319 - val_loss: 1.8065 - val_acc: 0.3375
Epoch 7/130
50000/50000 [==============================] - 205s 4ms/sample - loss: 1.7983 - acc: 0.3405 - val_loss: 1.8364 - val_acc: 0.3254
Epoch 8/130
50000/50000 [

Epoch 59/130
50000/50000 [==============================] - 211s 4ms/sample - loss: 1.5593 - acc: 0.4331 - val_loss: 1.5958 - val_acc: 0.4206
Epoch 60/130
50000/50000 [==============================] - 212s 4ms/sample - loss: 1.5610 - acc: 0.4340 - val_loss: 1.6093 - val_acc: 0.4127
Epoch 61/130
50000/50000 [==============================] - 211s 4ms/sample - loss: 1.5687 - acc: 0.4304 - val_loss: 1.5887 - val_acc: 0.4211
Epoch 62/130
50000/50000 [==============================] - 213s 4ms/sample - loss: 1.5652 - acc: 0.4332 - val_loss: 1.5744 - val_acc: 0.4256
Epoch 63/130
50000/50000 [==============================] - 211s 4ms/sample - loss: 1.5631 - acc: 0.4330 - val_loss: 1.5751 - val_acc: 0.4274
Epoch 64/130
50000/50000 [==============================] - 213s 4ms/sample - loss: 1.5607 - acc: 0.4347 - val_loss: 1.5758 - val_acc: 0.4222
Epoch 65/130
50000/50000 [==============================] - 212s 4ms/sample - loss: 1.5675 - acc: 0.4325 - val_loss: 1.5806 - val_acc: 0.4290
Epoch 

KeyboardInterrupt: 

### Model4: Custom ResNet

In [12]:
# Hyper-parameters
# epochs = 132
# batch_size = 500
# l2_norm_clip = 1.2
# noise_multiplier = 1.3
# num_microbatches = 100
# learning_rate = 2.5e-1
# delta = 1e-5
epochs = 55
batch_size = 500
l2_norm_clip = 2
noise_multiplier = 1
num_microbatches = 50
learning_rate = 1.3
delta = 1e-5

if batch_size % num_microbatches != 0:
    raise ValueError('Batch size should be an integer multiple of the number of microbatches')
    
# Compute RDP
orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
rdp = compute_rdp(q = batch_size / 50000,
                  noise_multiplier = noise_multiplier,
                  steps = epochs * 50000 // batch_size,
                  orders = orders)
# Calculate epsilon
epsilon = get_privacy_spent(orders, rdp, target_delta = delta)[0]
epsilon

4.830398168750132

In [13]:
### Custom ResNet model
model_input = Input(shape = (32, 32, 3))
x1 = Conv2D(64, (3, 3), activation = 'relu', strides = 2)(model_input)
x2 = MaxPooling2D(pool_size = (3,3))(x1)
## Residual Block 1
x3 = Conv2D(64, (3, 3), activation = 'relu', strides = 1)(x2)
x3 = GroupNormalization(groups = 64, axis=3)(x3) # normalization layer 1
x4 = Conv2D(64, (3, 3), activation = 'relu', strides = 1)(x3)
sk1 = Add()([x2, x4]) # skip connection 1
## Residual Block 2
x5 = Conv2D(64, (3, 3), activation = 'relu', strides = 1)(sk1)
x5 = GroupNormalization(groups = 64, axis=3)(x5) # normalization layer 2
x6 = Conv2D(64, (3, 3), activation = 'relu', strides = 1)(x5)
sk2 = Add()([sk1, x6]) # skip connection 2
# ## Residual Block 3
# x7 = Conv2D(64, (3, 3), activation = 'relu', strides = 1)(sk2)
# x7 = GroupNormalization(groups = 64, axis=3)(x7) # normalization layer 3
# x8 = Conv2D(64, (3, 3), activation = 'tanh', strides = 1)(x7)
# sk3 = Add()([sk2, x8]) # skip connection 3
# ## Residual Block 4
# x9 = Conv2D(64, (3, 3), activation = 'relu', strides = 1)(sk3)
# x9 = GroupNormalization(groups = 64, axis=3)(x9) # normalization layer 4
# x10 = Conv2D(64, (3, 3), activation = 'tanh', strides = 1)(x9)
# sk4 = Add()([sk3, x10]) # skip connection 4
# ## Residual Block 5
# x11 = Conv2D(64, (3, 3), activation = 'relu', strides = 1)(sk4)
# x11 = GroupNormalization(groups = 64, axis=3)(x11) # normalization layer 5
# x12 = Conv2D(64, (3, 3), activation = 'tanh', strides = 1)(x11)
# sk5 = Add()([sk4, x12]) # skip connection 5
# Global Average Pooling layer
# avg = GlobalAveragePooling2D()(sk2)
ftn = Flatten()(sk2)
# Feed-forward network
d1 = Dense(64, activation = 'relu')(ftn)
d2 = Dense(32, activation = 'relu')(d1)
model_output = Dense(10, activation = 'softmax')(d2)
model = Model(inputs = model_input, outputs = model_output)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_10 (Conv2D)             (None, 15, 15, 64)   1792        ['input_3[0][0]']                
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 5, 5, 64)    0           ['conv2d_10[0][0]']              
                                                                                                  
 conv2d_11 (Conv2D)             (None, 3, 3, 64)     36928       ['max_pooling2d_2[0][0]']        
                                                                                            

In [15]:
# define optimizer (dp-sgd) 
optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)

# # define optimizer (dp-adam)
# optimizer = tensorflow_privacy.DPKerasAdamOptimizer(
#     l2_norm_clip=l2_norm_clip,
#     noise_multiplier=noise_multiplier,
#     num_microbatches=num_microbatches,
#     learning_rate=learning_rate,
#     gradient_accumulation_steps=5)

# define loss function
loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

In [ ]:
# Compile model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
# Train model
model.fit(x_train, y_train,
          epochs=epochs,
          validation_data=(x_test, y_test),
          batch_size=batch_size)

Train on 50000 samples, validate on 10000 samples


2022-03-23 13:18:10.477792: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-23 13:18:11.304313: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1/55


2022-03-23 13:18:11.596346: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-23 13:18:11.739659: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50000/50000 [==============================] - ETA: 0s - loss: 4.1855 - acc: 0.1007

2022-03-23 13:26:40.755324: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50000/50000 [==============================] - 510s 10ms/sample - loss: 4.1855 - acc: 0.1007 - val_loss: 3.8759 - val_acc: 0.1002
Epoch 2/55
20500/50000 [===========>..................] - ETA: 4:24 - loss: 4.4127 - acc: 0.0964

### Model3: Custom CNN

In [3]:
# Hyper-parameters
epochs = 63
batch_size = 1000
l2_norm_clip = 1
noise_multiplier = 1.3
num_microbatches = 125
learning_rate = 0.25
delta = 1e-5

if batch_size % num_microbatches != 0:
    raise ValueError('Batch size should be an integer multiple of the number of microbatches')
    
# Compute RDP
orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
rdp = compute_rdp(q = batch_size / 50000,
                  noise_multiplier = noise_multiplier,
                  steps = epochs * 50000 // batch_size,
                  orders=orders)
# Calculate epsilon
epsilon = get_privacy_spent(orders, rdp, target_delta=delta)[0]
epsilon

4.959247448597203

In [4]:
# Define model
model = tf.keras.Sequential([
    
    tf.keras.layers.Conv2D(64, 5, strides=1, padding='same', activation='tanh', input_shape=(32, 32, 3)),
#     tf.keras.layers.ReLU(max_value = 50, negative_slope = 1.2),
#     tfa.layers.GroupNormalization(groups=32, axis=3),
    tf.keras.layers.MaxPool2D(2, 2),
    
    tf.keras.layers.Conv2D(32, 5, strides=1, padding='valid', activation='relu'),
    tfa.layers.GroupNormalization(groups=32, axis=3),
#     tf.keras.layers.ReLU(max_value = 50, negative_slope = 1.2),
    tf.keras.layers.MaxPool2D(2, 2),
    
    tf.keras.layers.Conv2D(32, 3, strides=2, padding='valid', activation='relu'),
    tfa.layers.GroupNormalization(groups=32, axis=3),
#     tf.keras.layers.ReLU(max_value = 50, negative_slope = 1.2),
    tf.keras.layers.MaxPool2D(2, 2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='tanh'),
#     tfa.layers.GroupNormalization(groups=64, axis=3),
#     tf.keras.layers.ReLU(max_value = 50, negative_slope = 1.2),
    tf.keras.layers.Dense(64, activation='tanh'),
    tf.keras.layers.Dense(32, activation='tanh'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        4864      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 12, 32)        51232     
                                                                 
 group_normalization (GroupN  (None, 12, 12, 32)       64        
 ormalization)                                                   
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 32)         0         
 2D)                                                             
                                                        

In [5]:
# define optimizer (dp-sgd) 
optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)

# # define optimizer (dp-adam)
# optimizer = tensorflow_privacy.DPKerasAdamOptimizer(
#     l2_norm_clip, noise_multiplier, num_microbatches)

# define loss function
loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

In [6]:
# Compile model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
# Train model
model.fit(x_train, y_train,
          epochs=epochs,
          validation_data=(x_test, y_test),
          batch_size=batch_size)

/Users/Mahos/miniforge3/envs/shm_tf/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


Train on 50000 samples, validate on 10000 samples
Metal device set to: Apple M1 Pro


2022-03-21 09:01:09.300782: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-21 09:01:09.300907: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-03-21 09:01:09.825404: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-21 09:01:09.837009: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-21 09:01:10.420292: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-21 09:01:10.590297: I tensorflow/core/grappler/o

Epoch 1/63


2022-03-21 09:01:10.701318: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50000/50000 [==============================] - ETA: 0s - loss: 2.3113 - acc: 0.1161 

/Users/Mahos/miniforge3/envs/shm_tf/lib/python3.9/site-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
2022-03-21 09:13:25.384671: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50000/50000 [==============================] - 736s 15ms/sample - loss: 2.3113 - acc: 0.1161 - val_loss: 2.2828 - val_acc: 0.1416
Epoch 2/63
50000/50000 [==============================] - 652s 13ms/sample - loss: 2.2807 - acc: 0.1360 - val_loss: 2.2638 - val_acc: 0.1461
Epoch 3/63
50000/50000 [==============================] - 641s 13ms/sample - loss: 2.2497 - acc: 0.1597 - val_loss: 2.2371 - val_acc: 0.1724
Epoch 4/63
50000/50000 [==============================] - 656s 13ms/sample - loss: 2.2215 - acc: 0.1786 - val_loss: 2.2130 - val_acc: 0.1821
Epoch 5/63
50000/50000 [==============================] - 654s 13ms/sample - loss: 2.2000 - acc: 0.1910 - val_loss: 2.1887 - val_acc: 0.2021
Epoch 6/63
50000/50000 [==============================] - 646s 13ms/sample - loss: 2.1910 - acc: 0.1901 - val_loss: 2.1810 - val_acc: 0.1983
Epoch 7/63
50000/50000 [==============================] - 649s 13ms/sample - loss: 2.1744 - acc: 0.1984 - val_loss: 2.1796 - val_acc: 0.1921
Epoch 8/63
50000/50000 [

KeyboardInterrupt: 

### Model1: Simple CNN

In [11]:
# Hyper-parameters
epochs = 20
batch_size = 500

l2_norm_clip = 1.5
noise_multiplier = 1.3
num_microbatches = 250
learning_rate = 0.25

if batch_size % num_microbatches != 0:
    raise ValueError('Batch size should be an integer multiple of the number of microbatches')

In [12]:
# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 8,
                           strides=2,
                           padding='same',
                           activation='relu',
                           input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Conv2D(32, 4,
                           strides=2,
                           padding='valid',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [13]:
# define optimizer (dp-sgd) 
optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)
# define loss function
loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

In [14]:
# Compile model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
# Train model
model.fit(x_train, y_train,
          epochs=epochs,
          validation_data=(x_test, y_test),
          batch_size=batch_size)

Train on 50000 samples, validate on 10000 samples


2022-03-08 16:42:30.484258: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-08 16:42:31.402756: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1/20


2022-03-08 16:42:31.686466: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-08 16:42:31.846137: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50000/50000 [==============================] - ETA: 0s - loss: 2.2188 - acc: 0.1664 

2022-03-08 17:04:09.816648: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50000/50000 [==============================] - 1299s 26ms/sample - loss: 2.2188 - acc: 0.1664 - val_loss: 2.1000 - val_acc: 0.2320
Epoch 2/20
50000/50000 [==============================] - 1223s 24ms/sample - loss: 2.0589 - acc: 0.2534 - val_loss: 1.9255 - val_acc: 0.3094
Epoch 3/20
50000/50000 [==============================] - 1212s 24ms/sample - loss: 1.9223 - acc: 0.3040 - val_loss: 1.8572 - val_acc: 0.3305
Epoch 4/20
50000/50000 [==============================] - 1217s 24ms/sample - loss: 1.8465 - acc: 0.3341 - val_loss: 1.8674 - val_acc: 0.3170
Epoch 5/20
50000/50000 [==============================] - 1208s 24ms/sample - loss: 1.7838 - acc: 0.3603 - val_loss: 1.7684 - val_acc: 0.3776
Epoch 6/20
50000/50000 [==============================] - 1221s 24ms/sample - loss: 1.7363 - acc: 0.3805 - val_loss: 1.7594 - val_acc: 0.3869
Epoch 7/20
50000/50000 [==============================] - 1227s 25ms/sample - loss: 1.6895 - acc: 0.4000 - val_loss: 1.7437 - val_acc: 0.3961
Epoch 8/20
50000/

In [15]:
# Calculate privacy performance
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=x_train.shape[0],
                                              batch_size=batch_size,
                                              noise_multiplier=noise_multiplier,
                                              epochs=epochs,
                                              delta=1e-5)

DP-SGD with sampling rate = 1% and noise_multiplier = 1.3 iterated over 2000 steps satisfies differential privacy with eps = 1.8 and delta = 1e-05.
The optimal RDP order is 11.0.


(1.798929515280597, 11.0)

### Model2: VGG with transfer learning

In [3]:
# Hyper-parameters
epochs = 50
batch_size = 1250

l2_norm_clip = 1.5
noise_multiplier = 1.3
num_microbatches = 250
learning_rate = 0.25

if batch_size % num_microbatches != 0:
    raise ValueError('Batch size should be an integer multiple of the number of microbatches')

In [4]:
#Building a VGG model with pretrained weights

vgg_model=tf.keras.Sequential()

#https://keras.io/api/applications/vgg/#vgg16-function
#https://keras.io/guides/transfer_learning/
#https://chroniclesofai.com/transfer-learning-with-keras-resnet-50/
pre_vgg_model=tf.keras.applications.VGG16(include_top=False,weights="imagenet",
                                          classes=10,pooling=max,input_shape=(32,32,3))
pre_vgg_model.trainable=False
vgg_model.add(pre_vgg_model)
vgg_model.add(tf.keras.layers.Flatten())
vgg_model.add(tf.keras.layers.Dense(512, activation='tanh'))
vgg_model.add(tf.keras.layers.Dense(10, activation='softmax'))


Metal device set to: Apple M1 Pro


2022-03-14 00:07:48.042660: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-14 00:07:48.042815: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-03-14 00:07:48.050509: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-14 00:07:48.050609: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-14 00:07:48.065426: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-14 00:07:48.153230: I tensorflow/core/grappler/o

In [5]:
vgg_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 14,982,474
Trainable params: 267,786
Non-trainable params: 14,714,688
_________________________________________________________________


In [6]:
# define optimizer (dp-sgd) 
vgg_optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)
# define loss function
vgg_loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

In [7]:
# Compile model
vgg_model.compile(optimizer=vgg_optimizer, loss=vgg_loss, metrics=['accuracy'])
# Train model
vgg_model.fit(x_train, y_train,
          epochs=epochs,
          validation_data=(x_test, y_test),
          batch_size=batch_size)

/Users/Mahos/miniforge3/envs/shm_tf/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


Train on 50000 samples, validate on 10000 samples


2022-03-14 00:08:06.080186: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1/50


2022-03-14 00:08:06.388985: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-14 00:08:06.466079: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-14 00:08:06.528768: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50000/50000 [==============================] - ETA: 0s - loss: 2.0519 - acc: 0.2842

/Users/Mahos/miniforge3/envs/shm_tf/lib/python3.9/site-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
2022-03-14 00:10:55.361394: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50000/50000 [==============================] - 174s 3ms/sample - loss: 2.0519 - acc: 0.2842 - val_loss: 1.8509 - val_acc: 0.3751
Epoch 2/50
50000/50000 [==============================] - 158s 3ms/sample - loss: 1.7477 - acc: 0.4120 - val_loss: 1.6785 - val_acc: 0.4330
Epoch 3/50
50000/50000 [==============================] - 157s 3ms/sample - loss: 1.6229 - acc: 0.4513 - val_loss: 1.5985 - val_acc: 0.4520
Epoch 4/50
50000/50000 [==============================] - 158s 3ms/sample - loss: 1.5563 - acc: 0.4692 - val_loss: 1.5471 - val_acc: 0.4685
Epoch 5/50
50000/50000 [==============================] - 156s 3ms/sample - loss: 1.5081 - acc: 0.4849 - val_loss: 1.5045 - val_acc: 0.4841
Epoch 6/50
50000/50000 [==============================] - 158s 3ms/sample - loss: 1.4763 - acc: 0.4935 - val_loss: 1.4800 - val_acc: 0.4906
Epoch 7/50
50000/50000 [==============================] - 160s 3ms/sample - loss: 1.4505 - acc: 0.5006 - val_loss: 1.4585 - val_acc: 0.4972
Epoch 8/50
50000/50000 [=======

In [8]:
# noise_multiplier = FLAGS.noise_multiplier
# sampling_probability = FLAGS.batch_size / 60000
# steps = FLAGS.epochs * 60000 // FLAGS.batch_size

In [9]:
# from absl import app
# from absl import flags
# from absl import logging
# FLAGS = flags.FLAGS
orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
rdp = compute_rdp(q = batch_size / 50000,
                  noise_multiplier = noise_multiplier,
                  steps = epochs * 50000 // batch_size,
                  orders=orders)

In [10]:
epsilon = get_privacy_spent(orders, rdp, target_delta=1e-5)[0]
epsilon

4.980956406486334